In [1]:
#Import modules
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import re 
import numpy as np


In [2]:
#See if scraping the website is legal
wikiurl="https://runescape.wiki/w/Sack_of_very_wild_rewards"
response=requests.get(wikiurl)
print('Go ahead and scrape') if response.status_code == 200 else print('Scraping this is illegal')

Go ahead and scrape


In [3]:
#Turn data from HTML into BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [4]:
#Creates dataframe from BeautifulSoup data, removes bottom row
df=pd.read_html(str(indiatable))
df=pd.DataFrame(df[0])

if "Based" in df.iloc[-1][0]:
    df = df.iloc[:-1 , :]
    print("Dropping droplog project row...")
df = df[['Item', 'Quantity', 'Rarity']]
df = df.drop(['Unnamed: 0', 'GE price', 'High Alch', 'Unnamed: 6'], axis = 1)
pd.options.display.float_format = '{:,.3f}'.format

#Function to average each string in a list for each row
def average_list(list):
    total = []
    for items in list:
        total.append(int(items))
    return np.average(total)

#Convert a fraction inside a string to a decimal
def string_fraction(fraction):
    num, dem = fraction.split('/')
    return(float(num) / float(dem))

#Clean list by removing unnessary words brackets, removing commas, removing ranges, and averaging the quantities
df['Quantity'] = df['Quantity'].apply(lambda x: re.sub(r"\(.*\)", "", x))
df['Quantity'] = df['Quantity'].str.replace(',',"")
df['Quantity'] = df['Quantity'].str.split('–')
df['Average Quantity'] = df['Quantity'].apply(lambda x: average_list(x))
df['Rarity'] = df['Rarity'].apply(lambda x: string_fraction(x))
df['Weighted Quantity'] = df['Rarity'] * df['Average Quantity']
df

Dropping droplog project row...


,Item,Quantity,Rarity,Average Quantity,Weighted Quantity
0,Coins,"[450000, 750000]",0.083,"600,000.000","49,688.482"
1,Dark onyx core,[1],0.002,1.000,0.002
2,Magic logs,"[250, 300 ]",0.078,275.000,21.436
3,Bakriminel bolts,"[100, 150]",0.074,125.000,9.293
4,Draconic visage,[1],0.020,1.000,0.020
5,Black dragon egg (unchecked),[1],0.084,1.000,0.084
6,Dragon Rider boots,[1],0.089,1.000,0.089
7,Dragon Rider gloves,[1],0.087,1.000,0.087
8,Large blunt necronium salvage,"[1, 3 ]",0.076,2.000,0.153
9,Huge spiky necronium salvage,"[1, 2 ]",0.091,1.500,0.136


In [5]:
#Selecting certain number of instances (Kills, opens etc)
#Default amount is 1
#Includes optional multiplier (default 1) for instances which give multiple rewards
def lootFromInstances(value = 1, multiplier = 1):
    temp_df = df[['Item']].copy()
    if multiplier == 1:
        temp_df[f'Expected amount in {value} instances'] = df['Weighted Quantity'] * value * multiplier
    else:
        temp_df[f'Expected amount in {value} instances, multiplier = {multiplier} '] = df['Weighted Quantity'] * value * multiplier
    return temp_df

#Combined loot when going on drop rate for a specific item
#Default is first item (usually coins)
#Includes potential multiplier (default 1) for instances which give multiple rewards
def lootFromDesiredItem(des_item = df['Item'][0], multiplier = 1):
    #Get index of specific item
    des_item = df['Item'][df['Item'] == des_item]
    #Creates a temporary copy dataframe
    temp_df = df[['Item']].copy()
    #Calculates number of instances per item
    reciprocal = float(1 / df['Weighted Quantity'][des_item.index]) 
    print(f"Number of instances needed for {des_item.values[0]} - " + str(reciprocal / multiplier))
    if multiplier != 1:
        print(f"(Assumes {multiplier} loots per instance)")
    temp_df[f'Expected combined loot per {des_item.values[0]}'] = reciprocal * df['Weighted Quantity']
    return temp_df


In [6]:
lootFromInstances()

,Item,Expected amount in 1 instances
0,Coins,"49,688.482"
1,Dark onyx core,0.002
2,Magic logs,21.436
3,Bakriminel bolts,9.293
4,Draconic visage,0.020
5,Black dragon egg (unchecked),0.084
6,Dragon Rider boots,0.089
7,Dragon Rider gloves,0.087
8,Large blunt necronium salvage,0.153
9,Huge spiky necronium salvage,0.136


In [7]:
lootFromDesiredItem()

Number of instances needed for Coins - 2.0125388546441177e-05


,Item,Expected combined loot per Coins
0,Coins,1.000
1,Dark onyx core,0.000
2,Magic logs,0.000
3,Bakriminel bolts,0.000
4,Draconic visage,0.000
5,Black dragon egg (unchecked),0.000
6,Dragon Rider boots,0.000
7,Dragon Rider gloves,0.000
8,Large blunt necronium salvage,0.000
9,Huge spiky necronium salvage,0.000
